# Librerías

In [249]:
''' 
    * Autor: Octavio Augusto Alemán Esparza
    * Fecha: 1.11.2023
    * Archivo: metadata.ipynb
    * Descripción: Notebook para la creación de los metadatos Días hasta liderazgo y Posiciones hasta liderazgo.
'''
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Carga de Datasets

In [250]:
df_res_his = pd.read_csv('./data/raw_data/Historico-Renuncias-Liverpool.csv')
df_res_his.head(5)

,Unnamed: 0,Nº pers.,Hasta,Desde,Soc.,Desc Soc,DivP,Desc Div Per,GrPer,Desc Gpo Personal,ÁPers,Desc APers,Clave de organización,Ubicación,Un.org.,Desc Un Org,Función,Desc Función
0,0,3369,1998-12-31 00:00:00,1974-12-01 00:00:00,140,Servicios Liverpool S.A.,SELI,Servicios Liverpool,B,Planta No Sind.,1E,Ejecutivo/ Coord.,540,JUNIORS,0,Sin Un Org,0,Sin Función
1,1,3369,1999-12-31 00:00:00,1999-01-01 00:00:00,140,Servicios Liverpool S.A.,SELI,Servicios Liverpool,B,Planta No Sind.,1E,Ejecutivo/ Coord.,540,JUNIORS,40,Compras Juniors,83,Comprador Sr B
2,2,3369,2004-01-31 00:00:00,2000-01-01 00:00:00,140,Servicios Liverpool S.A.,SELI,Servicios Liverpool,B,Planta No Sind.,1E,Ejecutivo/ Coord.,540,JUNIORS,40,Compras Juniors,83,Comprador Sr B
3,3,3369,2005-11-30 00:00:00,2004-02-01 00:00:00,140,Servicios Liverpool S.A.,SELI,Servicios Liverpool,B,Planta No Sind.,1E,Ejecutivo/ Coord.,540,JUNIORS,40,Compras Juniors,213,Comprador Sr A
4,4,3369,2013-03-31 00:00:00,2005-12-01 00:00:00,140,Servicios Liverpool S.A.,SELI,Servicios Liverpool,B,Planta No Sind.,1E,Ejecutivo/ Coord.,553,MUJER ACTUAL,63,Compras Mujer Actual,213,Comprador Sr A


In [251]:
df_res_dem = pd.read_csv('./data/raw_data/Demograficos-Renuncias-Liverpool (2).csv')
df_res_dem.head(5)

,Unnamed: 0,Nº pers.,Fecha nacimiento,Genero,No Hijos,Fecha ingreso,Edad ingreso,CP Vivienda,CP Trabajo,Ubicación,...,Fecha Salida,Antigüedad,Edad salida,Año salida,Desc Medida,Gpo Per,Desc Gpo Pers,Ultima Evaluación,Función,Desc Fun
0,0,14742898,1989-06-12,Hombre,1,2018-10-10,29,1100,1140,98,...,2019-01-01,0,29,2019,Baja,F,Eventual No Sind.,3.0,458,Auxiliar Armador
1,1,14691625,1993-07-26,Hombre,0,2018-09-06,25,1120,1140,98,...,2019-01-01,0,25,2019,Baja,D,Vía Planta No Sind.,3.0,919,Operador Camioneta
2,2,14714343,1978-10-04,Mujer,3,2018-09-24,39,1140,1140,98,...,2019-01-01,0,40,2019,Baja,F,Eventual No Sind.,3.0,927,Inteligencia Operativa
3,3,14790783,1996-10-20,Hombre,0,2018-11-12,22,1140,1140,98,...,2019-01-01,0,22,2019,Baja,F,Eventual No Sind.,3.0,966,Personal Operativo
4,4,14701213,2000-08-23,Hombre,0,2018-09-12,18,1160,1140,98,...,2019-01-01,0,18,2019,Baja,D,Vía Planta No Sind.,3.0,966,Personal Operativo


# Limpieza de datos
-- Se deben convertir las columnas a formato de fecha antes de usar las funciones.

-- Se deben eliminar años inválidos previamente.

In [252]:
# Convierte las columnas 'Desde' y 'Hasta' a formato de fecha, forzando la conversión y estableciendo valores nulos en caso de errores
df_res_his['Desde'] = pd.to_datetime(df_res_his['Desde'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_res_his['Hasta'] = pd.to_datetime(df_res_his['Hasta'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Elimina las filas con valores nulos en las columnas 'Desde' o 'Hasta'
df_res_his.dropna(subset=['Desde', 'Hasta'], inplace=True)

# Funciones para la obteción de metadatos
-- Días hasta posición de Liderazgo

-- Posiciones hasta liderazgo

Posiciones clave

In [253]:
keywords = ['Ejecutivo', 
            'Director', 
            'Gerente', 
            'Jefe', 
            'Administrador', 
            'Responsable', 
            'Encargado', 
            'Subdirector']

In [254]:
# Función para calcular los días hasta el primer puesto de liderazgo
def days_to_first_leadership(group):

    days = 0

    for index, row in group.iterrows():
        if any(keyword in row['Función'] for keyword in keywords):
            return days
        days += (row['Hasta'] - row['Desde']).days

    return -1

In [255]:
# Función para calcular las posiciones antes de alcanzar un puesto de liderazgo
def positions_before_leadership(group):

    positions = 1

    for index, row in group.iterrows():
        if any(keyword in row['Función'] for keyword in keywords):
            return positions
        positions += 1

    return -1

In [256]:
# Función integrada para la obtención de los metadatos e integración de los mismos en el dataframe demografico original
def getLeadership(df_demo, df_his, column):
    ''' 
        Recibe como parametros:
        df_demo: Dataframe demográfico original o Dataframe Combinado
        df_his: Dataframe Histrórico original
        column: Nombre de la columna de función dentro del dataframe histórico utilizado

        Regresa:
        target: dataframe demografico  o Dataframe Combinado con las columnas Días hasta liderazgo y Posiciones hasta liderazgo ya integradas.
    '''
    
    target = df_demo.copy()

    # Calcular los días hasta el primer puesto de liderazgo
    df_aux = df_his.copy()
    df_aux.rename(columns={column: 'Función'}, inplace=True)
    results_days = df_aux.groupby('Nº pers.').apply(days_to_first_leadership).reset_index(name='Días hasta liderazgo')

    # Calcular las posiciones hasta un puesto de liderazgo
    df_aux = df_his.copy()
    df_aux.rename(columns={column: 'Función'}, inplace=True)
    results_pos = df_aux.groupby('Nº pers.').apply(positions_before_leadership).reset_index(name='Posiciones hasta liderazgo')

    # Combinar los resultados con el dataframe original usando 'Nº pers' como clave de ordenamiento
    target = target.merge(results_days, on='Nº pers.', how='left')
    target = target.merge(results_pos, on='Nº pers.', how='left')

    return target



Visualización de resultados

In [257]:
# Ejemplo de llamada a la función
df_res_dem = getLeadership(df_res_dem, df_res_his, 'Desc Función')
df_res_dem

,Unnamed: 0,Nº pers.,Fecha nacimiento,Genero,No Hijos,Fecha ingreso,Edad ingreso,CP Vivienda,CP Trabajo,Ubicación,...,Edad salida,Año salida,Desc Medida,Gpo Per,Desc Gpo Pers,Ultima Evaluación,Función,Desc Fun,Días hasta liderazgo,Posiciones hasta liderazgo
0,0,14742898,1989-06-12,Hombre,1,2018-10-10,29,1100,1140,98,...,29,2019,Baja,F,Eventual No Sind.,3.00,458,Auxiliar Armador,-1.0,-1.0
1,1,14691625,1993-07-26,Hombre,0,2018-09-06,25,1120,1140,98,...,25,2019,Baja,D,Vía Planta No Sind.,3.00,919,Operador Camioneta,-1.0,-1.0
2,2,14714343,1978-10-04,Mujer,3,2018-09-24,39,1140,1140,98,...,40,2019,Baja,F,Eventual No Sind.,3.00,927,Inteligencia Operativa,-1.0,-1.0
3,3,14790783,1996-10-20,Hombre,0,2018-11-12,22,1140,1140,98,...,22,2019,Baja,F,Eventual No Sind.,3.00,966,Personal Operativo,-1.0,-1.0
4,4,14701213,2000-08-23,Hombre,0,2018-09-12,18,1160,1140,98,...,18,2019,Baja,D,Vía Planta No Sind.,3.00,966,Personal Operativo,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117155,117155,14303266,1996-08-01,Hombre,0,2017-10-11,21,90300,5348,695,...,26,2023,Baja,D,Vía Planta No Sind.,3.00,0,Sin Función,-1.0,-1.0
117156,117156,15776651,1997-07-06,Hombre,2,2022-04-21,24,37545,37545,94,...,25,2023,Baja,B,Planta No Sind.,3.00,919,Operador Camioneta,-1.0,-1.0
117157,117157,15328942,1993-09-22,Hombre,2,2020-08-31,26,37549,37545,94,...,29,2023,Baja,B,Planta No Sind.,3.16,901,Operador Camion,-1.0,-1.0
117158,117158,15930226,1983-12-31,Hombre,1,2022-10-03,38,54725,5348,9002,...,39,2023,Baja,D,Vía Planta No Sind.,3.00,6002,Coordinador Relaciones Si,-1.0,-1.0
